In [1]:
!pip install PyGithub

In [130]:
from github import Github
import os
import base64
import pandas as pd
import numpy as np
import re

In [2]:
with open("github.txt") as myfile:
    firstNlines=myfile.readlines()[0:2]
myfile.close()

In [3]:
g = Github(firstNlines[0].strip(), firstNlines[1])

In [5]:
repo=g.get_repo('CSSEGISandData/COVID-19')

In [7]:
contents = repo.get_contents("")

In [16]:
contents[3]

ContentFile(path="csse_covid_19_data")

In [22]:
def get_sha_for_tag(repository, tag):
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [66]:
def download_directory(repository, sha, server_path):
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    contents = repository.get_contents(server_path, ref=sha)

    for content in contents:
        print("Processing %s" % content.path)
        if content.type == 'dir':
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                file_data = base64.b64decode(file_content.content).decode('ascii')
                file_out = open(content.name, "w")
                file_out.write(file_data)
                file_out.close()
            except:
                pass

In [25]:
sha = get_sha_for_tag(repo, 'master')

In [67]:
download_directory(repo, sha, 'csse_covid_19_data/csse_covid_19_daily_reports')

Processing csse_covid_19_data/csse_covid_19_daily_reports/.gitignore
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-23-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-26-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-27-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-29-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-30-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-31-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-01-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-02-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_repor

In [69]:
df = pd.read_csv(r'C:\Users\brian\projects\COVID-19\csse_daily_reports\02-01-2020.csv')

In [73]:
from os import listdir
from os.path import isfile, join
mypath = r'C:\Users\brian\projects\COVID-19\csse_daily_reports\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [76]:
def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [95]:
nonzero_files = []
for file in onlyfiles:
    if is_non_zero_file(mypath + file) and file[-3:]=='csv':
        nonzero_files.append(mypath + file)

In [96]:
nonzero_files

['C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-01-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-02-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-03-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-04-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-05-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-06-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-07-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-08-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-09-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-10-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-11-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-12-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_r

In [245]:
df_list = []
for file in nonzero_files:
    df_1 = pd.read_csv(file)
    df_1['filename'] = file[-15:]
    df_list.append(df_1)  

In [257]:
df = pd.concat(df_list, ignore_index=True)

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3754 entries, 0 to 3753
Data columns (total 9 columns):
Confirmed         3754 non-null int64
Country/Region    3754 non-null object
Deaths            3754 non-null int64
Last Update       3754 non-null object
Latitude          1429 non-null float64
Longitude         1429 non-null float64
Province/State    2359 non-null object
Recovered         3754 non-null int64
filename          3754 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 264.1+ KB


In [258]:
df['Date'] = pd.to_datetime(df['Last Update'])

In [259]:
df.drop(columns=['Latitude', 'Longitude'], inplace=True)

In [260]:
df['Diamond Princess'] = df['Province/State'].str.contains('Diamond Princess')

In [194]:
df.loc[df['Province/State']=='From Diamond Princess']

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Date,Diamond Princess
1499,4,Australia,0,2020-02-21T23:03:13,From Diamond Princess,0,2020-02-21 23:03:13,True
1518,1,Israel,0,2020-02-21T15:33:03,From Diamond Princess,0,2020-02-21 15:33:03,True
1580,7,Australia,0,2020-02-22T17:03:05,From Diamond Princess,0,2020-02-22 17:03:05,True
1602,1,Israel,0,2020-02-22T20:53:02,From Diamond Princess,0,2020-02-22 20:53:02,True
2037,8,Australia,0,2020-02-27T03:33:06,From Diamond Princess,0,2020-02-27 03:33:06,True
2321,0,Australia,0,2020-02-29T02:03:10,From Diamond Princess,0,2020-02-29 02:03:10,True


In [261]:
df.drop(columns=['filename'], inplace=True)

In [262]:
df = df.drop_duplicates()

In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2106 entries, 0 to 3741
Data columns (total 8 columns):
Confirmed           2106 non-null int64
Country/Region      2106 non-null object
Deaths              2106 non-null int64
Last Update         2106 non-null object
Province/State      1435 non-null object
Recovered           2106 non-null int64
Date                2106 non-null datetime64[ns]
Diamond Princess    1435 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 148.1+ KB


In [311]:
df['Province/State'] = df['Province/State'].str.replace(' \(From Diamond Princess\)', '').replace('None', 'From Diamond Princess')

In [312]:
df['Province/State'].unique()

array(['Hubei', 'Zhejiang', 'Guangdong', 'Henan', 'Hunan', 'Anhui',
       'Jiangxi', 'Chongqing', 'Sichuan', 'Shandong', 'Jiangsu',
       'Shanghai', 'Beijing', 'Fujian', 'Shaanxi', 'Guangxi', 'Hebei',
       'Yunnan', 'Heilongjiang', 'Liaoning', 'Hainan', 'Shanxi',
       'Tianjin', 'Gansu', 'Guizhou', 'Ningxia', 'Inner Mongolia', nan,
       'Xinjiang', 'Jilin', 'Hong Kong', 'Taiwan', 'Qinghai', 'Macau',
       'New South Wales', 'Victoria', 'Queensland', 'Ontario',
       'Chicago, IL', 'South Australia', 'British Columbia', 'Tibet',
       'Boston, MA', 'Los Angeles, CA', 'Orange, CA', 'Santa Clara, CA',
       'Seattle, WA', 'Tempe, AZ', 'San Benito, CA', 'Toronto, ON',
       'London, ON', 'Madison, WI', 'Cruise Ship',
       'Diamond Princess cruise ship', 'San Diego County, CA',
       'San Antonio, TX', 'Ashland, NE', 'Travis, CA',
       'From Diamond Princess', 'Lackland, TX', 'Humboldt County, CA',
       'Sacramento County, CA', 'Omaha, NE', 'Unassigned Location',
      

In [316]:
df['Province/State'].unique()

array(['Hubei', 'Zhejiang', 'Guangdong', 'Henan', 'Hunan', 'Anhui',
       'Jiangxi', 'Chongqing', 'Sichuan', 'Shandong', 'Jiangsu',
       'Shanghai', 'Beijing', 'Fujian', 'Shaanxi', 'Guangxi', 'Hebei',
       'Yunnan', 'Heilongjiang', 'Liaoning', 'Hainan', 'Shanxi',
       'Tianjin', 'Gansu', 'Guizhou', 'Ningxia', 'Inner Mongolia', nan,
       'Xinjiang', 'Jilin', 'Hong Kong', 'Taiwan', 'Qinghai', 'Macau',
       'New South Wales', 'Victoria', 'Queensland', 'Ontario',
       'Chicago, IL', 'South Australia', 'British Columbia', 'Tibet',
       'Boston, MA', 'Los Angeles, CA', 'Orange, CA', 'Santa Clara, CA',
       'Seattle, WA', 'Tempe, AZ', 'San Benito, CA', 'Toronto, ON',
       'London, ON', 'Madison, WI', 'Cruise Ship',
       'Diamond Princess cruise ship', 'San Diego County, CA',
       'San Antonio, TX', 'Ashland, NE', 'Travis, CA',
       'From Diamond Princess', 'Lackland, TX', 'Humboldt County, CA',
       'Sacramento County, CA', 'Omaha, NE', 'Unassigned Location',
      